# Introduction

The U-Net model is a simple fully  convolutional neural network that is used for binary segmentation i.e foreground and background pixel-wise classification. Mainly, it consists of two parts. 

*   Contracting Path: we apply a series of conv layers and downsampling layers  (max-pooling) layers to reduce the spatial size 
*   Expanding Path: we apply a series of upsampling layers to reconstruct the spatial size of the input. 

The two parts are connected using a concatenation layers among different levels. This allows learning different features at different levels. At the end we have a simple conv 1x1 layer to reduce the number of channels to 1. 


![alt text](https://blog.playment.io/wp-content/uploads/2018/03/Screen-Shot-2018-09-05-at-3.00.03-PM.png)

# Imports

In [ ]:
!pip install tensorflow-addons

import numpy as np
import matplotlib.pyplot as plt
import scipy.io as sio
import imageio
import scipy.misc
import os
from PIL import Image
import keras
import tensorflow as tf
from keras import backend as K
from scipy.ndimage import distance_transform_edt as distance
import tensorflow.keras.optimizers as opt
from tensorflow.keras.models import Model
from tensorflow.keras.models import load_model as ldmd
from keras.layers import Conv2D, MaxPooling2D, Input, Conv2DTranspose, Concatenate, BatchNormalization, UpSampling2D
from keras.layers import  Dropout, Activation
import tensorflow_datasets as tfds
import tensorflow_addons as tfa
import cv2

from keras.layers.advanced_activations import LeakyReLU
from keras.callbacks import ModelCheckpoint, ReduceLROnPlateau, EarlyStopping
from keras import backend as K
from tensorflow.keras.utils import plot_model


# See if you need to mount the drive or something else
from google.colab import drive
drive.mount('./drive')

import glob
import random
import cv2
from random import shuffle

# Dataset

In [ ]:
# !zip -r ./drive/MyDrive/colab/data4test_256.zip ./drive/MyDrive/colab/data4test

# Generators

In [ ]:

def trainImage_generator(files, batch_size = 32, sz = (256, 256)):
  
  while True: 
    
    #extract a random batch 
    batch = np.random.choice(files, size = batch_size)    
    
    #variables for collecting batches of inputs and outputs 
    batch_x = []
    batch_y = []
    # batch_z = []
    
    
    for f in batch:




        #get the masks. Note that masks are png files 
        mask = Image.open(f'./drive/MyDrive/colab/data4test/label/{f[:-3]}'+"png")
        mask = np.array(mask)


        #preprocess the mask 
        # mask[mask >= 2] = 0 
        # mask[mask != 0 ] = 1
        


        #preprocess the raw images 
        # raw = Image.open(f'syntheticData/data/{f}')
        # raw = raw.resize(sz)
        # raw = np.array(raw)

        rawMat = sio.loadmat(f'./drive/MyDrive/colab/data4test/data/{f}')
        xData = np.array(rawMat['vxSample'])
        yData = np.array(rawMat['vySample'])

        ImgSize = xData.shape

        input_image = np.stack((xData,yData,np.zeros(xData.shape)), -1)
        # input_mask = np.stack((mask,mask,mask), -1)


        # input_mask = tf.convert_to_tensor(input_mask, np.float32)
        # input_image = tf.convert_to_tensor(input_image, np.float32)

        # # rotation in 30° steps
        # if random.random() > 0.5:
        #   rot_factor = tf.cast(tf.random.uniform(shape=[], maxval=12, dtype=tf.int32), tf.float32)
        #   angle = np.pi/12*rot_factor
        #   input_image = tfa.image.rotate(input_image, angle)
        #   input_mask = tfa.image.rotate(input_mask, angle)


        # if random.random() > 0.5:
        #     # use original image to preserve high resolution
        #     input_image = tf.image.central_crop(input_image, 0.75)
        #     input_mask = tf.image.central_crop(input_mask, 0.75)

        # resize
        input_image = cv2.resize(input_image, sz)
        input_mask = cv2.resize(mask, sz)

        # kernel = cv2.getStructuringElement(cv2.MORPH_ELLIPSE, (11,11))
        # input_mask = cv2.morphologyEx(input_mask,cv2.MORPH_OPEN, kernel)

        # flipping random horizontal or vertical
        if random.random() > 0.5:
            input_image = cv2.flip(input_image,0)
            input_mask = cv2.flip(input_mask,0)
        if random.random() > 0.5:
            input_image = cv2.flip(input_image,1)
            input_mask = cv2.flip(input_mask,1)

        # if random.random() > 0.5:
        #     xData = np.fliplr(xData)
        #     yData = np.fliplr(yData)
        #     mask = np.fliplr(mask)
        # if random.random() > 0.5:
        #     xData = np.flipud(xData)
        #     yData = np.flipud(yData)
        #     mask = np.flipud(mask)




        # input_mask = tf.image.rgb_to_grayscale(input_mask)

        input_image = np.asarray(input_image)
        input_mask = np.asarray(input_mask)
        input_mask[input_mask != 0 ] = 1

        batch_x.append(input_image)
        batch_y.append(input_mask)
        # batch_z.append(f)


    #preprocess a batch of images and masks 
    # batch_x = np.array(batch_x)/255.
    batch_x = np.array(batch_x)
    batch_y = np.array(batch_y)
    batch_y = np.expand_dims(batch_y,3)



    yield (batch_x, batch_y)     





def validImage_generator(files, batch_size = 32, sz = (256, 256)):
  
  while True: 
    
    #extract a random batch 
    batch = np.random.choice(files, size = batch_size)    
    
    #variables for collecting batches of inputs and outputs 
    batch_x = []
    batch_y = []
    # batch_z = []
    
    
    for f in batch:




        #get the masks. Note that masks are png files 
        mask = Image.open(f'./drive/MyDrive/colab/data4test/label/{f[:-3]}'+"png")
        mask = np.array(mask)


        #preprocess the mask 
        # mask[mask >= 2] = 0 
        # mask[mask != 0 ] = 1
        


        #preprocess the raw images 
        # raw = Image.open(f'syntheticData/data/{f}')
        # raw = raw.resize(sz)
        # raw = np.array(raw)

        rawMat = sio.loadmat(f'./drive/MyDrive/colab/data4test/data/{f}')
        xData = np.array(rawMat['vxSample'])
        yData = np.array(rawMat['vySample'])

        ImgSize = xData.shape

        input_image = np.stack((xData,yData,np.zeros(xData.shape)), -1)
        # input_mask = np.stack((mask,mask,mask), -1)


        # input_mask = tf.convert_to_tensor(input_mask, np.float32)
        # input_image = tf.convert_to_tensor(input_image, np.float32)

        # # rotation in 30° steps
        # if random.random() > 0.5:
        #   rot_factor = tf.cast(tf.random.uniform(shape=[], maxval=12, dtype=tf.int32), tf.float32)
        #   angle = np.pi/12*rot_factor
        #   input_image = tfa.image.rotate(input_image, angle)
        #   input_mask = tfa.image.rotate(input_mask, angle)


        # if random.random() > 0.5:
        #     # use original image to preserve high resolution
        #     input_image = tf.image.central_crop(input_image, 0.75)
        #     input_mask = tf.image.central_crop(input_mask, 0.75)

        # resize
        input_image = cv2.resize(input_image, sz)
        input_mask = cv2.resize(mask, sz)

        # kernel = cv2.getStructuringElement(cv2.MORPH_ELLIPSE, (11,11))
        # input_mask = cv2.morphologyEx(input_mask,cv2.MORPH_OPEN, kernel)

        input_image = np.asarray(input_image)
        input_mask = np.asarray(input_mask)
        input_mask[input_mask != 0 ] = 1

        batch_x.append(input_image)
        batch_y.append(input_mask)
        # batch_z.append(f)


    #preprocess a batch of images and masks 
    # batch_x = np.array(batch_x)/255.
    batch_x = np.array(batch_x)
    batch_y = np.array(batch_y)
    batch_y = np.expand_dims(batch_y,3)



    yield (batch_x, batch_y)   


    

In [ ]:
batch_size = 128
all_files = os.listdir('./drive/MyDrive/colab/data4test/data/')
shuffle(all_files)

split = int(0.85 * len(all_files))

#split into training and testing
train_files = all_files[0:split]
test_files  = all_files[split:]

train_generator = trainImage_generator(train_files, batch_size = batch_size)
test_generator  = validImage_generator(test_files, batch_size = batch_size)
final_test_generator = validImage_generator(all_files, batch_size = batch_size)

In [ ]:
x, y= next(train_generator)

In [ ]:
plt.axis('off')
img = x[0]
msk = y[0].squeeze()



msk = np.stack((msk,)*3, axis=-1)

print(msk.shape)
print(img.shape)


vel_img = np.sqrt(np.power(img[:,:,0],2)+np.power(img[:,:,1],2))

# vel_img = np.stack((vel_img,vel_img,vel_img), axis = -1)
plt.imshow( np.concatenate([img, msk], axis = 1))
# plt.imshow(msk)
plt.show()

# plt.imshow(img)
# plt.show


# print(z[0])
# print("img:",img)
# print("msk:",msk)
# print("img*msk:",img*msk)


mdict = {"vxSample": img[:,:,0], "vySample": img[:,:,1],"mask":msk}
sio.savemat("testimage.mat",mdict)


# IoU metric

The intersection over union (IoU) metric is a simple metric used to evaluate the performance of a segmentation algorithm. Given two masks $y_{true}, y_{pred}$ we evaluate 

$$IoU = \frac{y_{true} \cap y_{pred}}{y_{true} \cup y_{pred}}$$

In [ ]:
def mean_iou(y_true, y_pred):
    yt0 = K.cast(y_true[:,:,:,0],'float32')
    yp0 = K.cast(y_pred[:,:,:,0] > 0.5, 'float32')
    inter = tf.math.count_nonzero(tf.logical_and(tf.equal(yt0, 1), tf.equal(yp0, 1)))
    union = tf.math.count_nonzero(tf.add(yt0, yp0))
    iou = tf.where(tf.equal(union, 0), 1., tf.cast(inter/union, 'float32'))
    return iou

# def mean_iou(y_true, y_pred):

def IoULoss(y_true, y_pred, smooth=1e-6):
    
    #flatten label and prediction tensors
    inputs = K.flatten(K.cast(y_pred,'float32'))
    targets = K.flatten(K.cast(y_true,'float32'))
    

    intersection = K.sum(targets*inputs)
    total = K.sum(targets) + K.sum(inputs)
    union = total - intersection
    
    IoU = (intersection + smooth) / (union + smooth)
    return 1 - IoU




# Dice Related Loss

In [ ]:
def DiceLoss(targets, inputs, smooth=1e-6):
    
    #flatten label and prediction tensors
    inputs = K.flatten(K.cast(inputs,'float32'))
    targets = K.flatten(K.cast(targets,'float32'))
    
    intersection = K.sum(targets*inputs)
    dice = (2*intersection + smooth) / (K.sum(targets) + K.sum(inputs) + smooth)
    return 1 - dice





def DiceBCELoss(targets, inputs, smooth=1e-6):    
       
    #flatten label and prediction tensors
    inputs = K.flatten(K.cast(inputs,'float32'))
    targets = K.flatten(K.cast(targets,'float32'))
    
    BCE =  tf.keras.losses.binary_crossentropy(targets, inputs)
    intersection = K.sum(targets*inputs)   
    dice_loss = 1 - (2*intersection + smooth) / (K.sum(targets) + K.sum(inputs) + smooth)
    Dice_BCE = BCE + dice_loss
    
    return Dice_BCE

# Boundary Loss

In [ ]:
def calc_dist_map(seg):
    res = np.zeros_like(seg)
    posmask = seg.astype(bool)

    if posmask.any():
        negmask = ~posmask
        res = distance(negmask) * negmask - (distance(posmask) - 1) * posmask

    return res


def calc_dist_map_batch(y_true):
    y_true_numpy = y_true.numpy()
    return np.array([calc_dist_map(y)
                     for y in y_true_numpy]).astype(np.float32)


def surface_loss_keras(y_true, y_pred):
    y_true_dist_map = tf.py_function(func=calc_dist_map_batch,
                                     inp=[y_true],
                                     Tout=tf.float32)
    multipled = y_pred * y_true_dist_map
    return K.mean(multipled)

# Combination of loss function

In [ ]:
def BCEDiceIOU(targets, inputs, smooth=1e-6):    
       
    #flatten label and prediction tensors
    inputs = K.flatten(K.cast(inputs,'float32'))
    targets = K.flatten(K.cast(targets,'float32'))
    
    BCE =  tf.keras.losses.binary_crossentropy(targets, inputs)
    intersection = K.sum(targets*inputs)   
    dice_loss = 1 - (2*intersection + smooth) / (K.sum(targets) + K.sum(inputs) + smooth)


    total = K.sum(targets) + K.sum(inputs)
    union = total - intersection
    
    IoU = 1 - (intersection + smooth) / (union + smooth)

    Dice_BCE_IoU = BCE + dice_loss + IoU
    
    return Dice_BCE_IoU

# Other loss function

In [ ]:
def dsc(y_true, y_pred):
    smooth = 1.
    y_true_f = K.flatten(y_true)
    y_pred_f = K.flatten(y_pred)
    intersection = K.sum(y_true_f * y_pred_f)
    score = (2. * intersection + smooth) / (K.sum(y_true_f) + K.sum(y_pred_f) + smooth)
    return score



def confusion(y_true, y_pred):
    smooth=1
    y_pred_pos = K.clip(y_pred, 0, 1)
    y_pred_neg = 1 - y_pred_pos
    y_pos = K.clip(y_true, 0, 1)
    y_neg = 1 - y_pos
    tp = K.sum(y_pos * y_pred_pos)
    fp = K.sum(y_neg * y_pred_pos)
    fn = K.sum(y_pos * y_pred_neg) 
    prec = (tp + smooth)/(tp+fp+smooth)
    recall = (tp+smooth)/(tp+fn+smooth)
    return prec, recall

def tp(y_true, y_pred):
    smooth = 1
    y_pred_pos = K.round(K.clip(y_pred, 0, 1))
    y_pos = K.round(K.clip(y_true, 0, 1))
    tp = (K.sum(y_pos * y_pred_pos) + smooth)/ (K.sum(y_pos) + smooth) 
    return tp 

def tn(y_true, y_pred):
    smooth = 1
    y_pred_pos = K.round(K.clip(y_pred, 0, 1))
    y_pred_neg = 1 - y_pred_pos
    y_pos = K.round(K.clip(y_true, 0, 1))
    y_neg = 1 - y_pos 
    tn = (K.sum(y_neg * y_pred_neg) + smooth) / (K.sum(y_neg) + smooth )
    return tn 

def tversky(y_true, y_pred):
    smooth = 1
    y_true_pos = K.flatten(y_true)
    y_pred_pos = K.flatten(y_pred)
    true_pos = K.sum(y_true_pos * y_pred_pos)
    false_neg = K.sum(y_true_pos * (1-y_pred_pos))
    false_pos = K.sum((1-y_true_pos)*y_pred_pos)
    alpha = 0.7
    return (true_pos + smooth)/(true_pos + alpha*false_neg + (1-alpha)*false_pos + smooth)

def tversky_loss(y_true, y_pred):
    y_pred = K.flatten(K.cast(y_pred,'float32'))
    y_true = K.flatten(K.cast(y_true,'float32'))
    return 1 - tversky(y_true,y_pred)

def focal_tversky(y_true,y_pred):
    y_pred = K.flatten(K.cast(y_pred,'float32'))
    y_true = K.flatten(K.cast(y_true,'float32'))
    pt_1 = tversky(y_true, y_pred)
    gamma = 0.75
    return K.pow((1-pt_1), gamma)

# Model

In [ ]:
def unet(sz = (256, 256, 3)):
  x = Input(sz)
  inputs = x
  
  #down sampling 
  f = 8
  layers = []
  
  for i in range(0, 6):
    x = Conv2D(f, 3, activation='relu', padding='same') (x)
    x = Conv2D(f, 3, activation='relu', padding='same') (x)
    layers.append(x)
    x = MaxPooling2D() (x)
    f = f*2
  ff2 = 64 
  
  #bottleneck 
  j = len(layers) - 1
  x = Conv2D(f, 3, activation='relu', padding='same') (x)
  x = Conv2D(f, 3, activation='relu', padding='same') (x)
  x = Conv2DTranspose(ff2, 2, strides=(2, 2), padding='same') (x)
  x = Concatenate(axis=3)([x, layers[j]])
  j = j -1 
  
  #upsampling 
  for i in range(0, 5):
    ff2 = ff2//2
    f = f // 2 
    x = Conv2D(f, 3, activation='relu', padding='same') (x)
    x = Conv2D(f, 3, activation='relu', padding='same') (x)
    x = Conv2DTranspose(ff2, 2, strides=(2, 2), padding='same') (x)
    x = Concatenate(axis=3)([x, layers[j]])
    j = j -1 
    
  
  #classification 
  x = Conv2D(f, 3, activation='relu', padding='same') (x)
  x = Conv2D(f, 3, activation='relu', padding='same') (x)
  outputs = Conv2D(1, 1, activation='sigmoid') (x)
  

  # Decay learning rate
  initial_learning_rate = 0.0005
  lr_schedule = tf.keras.optimizers.schedules.ExponentialDecay(
          initial_learning_rate,
          decay_steps=800,
          decay_rate=0.96,
          staircase=True)

  #model creation 
  model = Model(inputs=[inputs], outputs=[outputs])
  rmsprop_optimizer = opt.RMSprop(learning_rate=lr_schedule, rho=0.9, epsilon=1e-08, decay=0.0)
  model.compile(optimizer = rmsprop_optimizer, loss = tversky_loss, metrics = [mean_iou])
  # model.compile(optimizer = 'rmsprop', loss = 'binary_crossentropy', metrics = [mean_iou])

  
  return model

In [ ]:
model = unet()
# model.load_weights("./drive/MyDrive/colab/unet.h5")

# Callbacks

Simple functions to save the model at each epoch and show some predictions 

In [ ]:
def build_callbacks():
        checkpointer = ModelCheckpoint(filepath='./drive/MyDrive/colab/unet.h5', verbose=0, save_best_only=True, save_weights_only=True)
        callbacks = [checkpointer, PlotLearning()]
        return callbacks

# inheritance for training process plot 
class PlotLearning(keras.callbacks.Callback):

    def on_train_begin(self, logs={}):
        self.i = 0
        self.x = []
        self.losses = []
        self.val_losses = []
        self.acc = []
        self.val_acc = []
        #self.fig = plt.figure()
        self.logs = []
    def on_epoch_end(self, epoch, logs={}):
        self.logs.append(logs)
        self.x.append(self.i)
        self.losses.append(logs.get('loss'))
        self.val_losses.append(logs.get('val_loss'))
        self.acc.append(logs.get('mean_iou'))
        self.val_acc.append(logs.get('val_mean_iou'))
        self.i += 1
        print('i=',self.i,'loss=',logs.get('loss'),'val_loss=',logs.get('val_loss'),'mean_iou=',logs.get('mean_iou'),'val_mean_iou=',logs.get('val_mean_iou'))
        
        f = open('./drive/MyDrive/colab/loss.txt', 'a')
        f.write(str(logs.get('loss'))+'\n')
        f.close()

        f = open('./drive/MyDrive/colab/val_loss.txt', 'a')
        f.write(str(logs.get('val_loss'))+'\n')
        f.close()

        f = open('./drive/MyDrive/colab/mean_iou.txt', 'a')
        f.write(str(logs.get('mean_iou'))+'\n')
        f.close()

        f = open('./drive/MyDrive/colab/val_mean_iou.txt', 'a')
        f.write(str(logs.get('val_mean_iou'))+'\n')
        f.close()


        #choose a random test image and preprocess
        path = np.random.choice(test_files)
        # raw = Image.open(f'syntheticData/trainingData/{path}')
        # raw = np.array(raw.resize((256, 256)))/255.
        # raw = raw[:,:,0:3]

        rawMat = sio.loadmat(f'./drive/MyDrive/colab/data4test/data/{path}')
        xData = np.array(rawMat['vxSample'])
        imgX = Image.fromarray(xData)
        xData = np.array(imgX.resize((256,256)))
        yData = np.array(rawMat['vySample'])
        imgY = Image.fromarray(yData)
        yData = np.array(imgY.resize((256,256)))
        raw = np.stack((xData, yData, np.zeros(xData.shape)), axis = -1)


        
        #predict the mask 
        pred = model.predict(np.expand_dims(raw, 0))
        
        #mask post-processing 
        msk  = pred.squeeze()
        msk = np.stack((msk,)*3, axis=-1)
        msk[msk >= 0.5] = 1 
        msk[msk < 0.5] = 0 
        
        groundTruth = Image.open(f'./drive/MyDrive/colab/data4test/label/{path[:-3]}png')

        #show the mask and the segmented image 
        combined = np.concatenate([raw, msk*255, raw* msk], axis = 1)
        plt.axis('off')
        plt.imshow(combined)
        plt.show()
        print(path[:-9])
        plt.axis('off')
        plt.imshow(groundTruth)
        plt.show()

# Training

In [ ]:
train_steps = len(train_files) //batch_size
test_steps = len(test_files) //batch_size



model.fit(train_generator, 
                    epochs = 400, steps_per_epoch = train_steps,validation_data = test_generator, validation_steps = test_steps,
                    callbacks = build_callbacks(), verbose = 0)


# Testing

In [ ]:
#Test file

test_files = sorted(os.listdir('./drive/MyDrive/colab/data4test/data'))
groundTruth_files = sorted(os.listdir('./drive/MyDrive/colab/data4test/label'))

#split into training and testing




#variables for collecting batches of inputs and outputs 
batch_x = []
batch_y = []



for i in range(0, len(test_files)):
# for i in range(0, 1):
  rawtTest = sio.loadmat('./drive/MyDrive/colab/data4test/data/'+test_files[i])
  rawGround = Image.open('./drive/MyDrive/colab/data4test/label/'+groundTruth_files[i]).resize((256,256))
  xData = np.array(rawtTest['vxSample'])
  imgX = Image.fromarray(xData)
  xData = np.array(imgX.resize((256,256)))
  yData = np.array(rawtTest['vySample'])
  imgY = Image.fromarray(yData)
  yData = np.array(imgY.resize((256,256)))
  raw = np.stack((xData, yData, np.zeros(xData.shape)), axis = -1)





  # raw = Image.open('385_33.png')
  # raw = np.array(raw.resize((256, 256)))/255.
  # raw = raw[:,:,0:3]

  #predict the mask 
  pred = model.predict(np.expand_dims(raw, 0))

  #mask post-processing 
  msk  = pred.squeeze()
  msk = np.stack((msk,)*3, axis=-1)
  msk[msk >= 0.5] = 1 
  msk[msk < 0.5] = 0 

  #show the mask and the segmented image 
  combined = np.concatenate([raw, msk, raw*msk], axis = 1)

  savePath = test_files[i]
  imageio.imwrite('./drive/MyDrive/colab/data4test/predict/'+savePath[:-4]+'.png', msk)
  # pred_Image = Image.fromarray(msk)
  # pred_Image.save('data4test/predict/'+savePath[:-4]+'predict.png')

  plt.axis('off')
  plt.imshow(combined)
  # plt.imshow(raw)
  # plt.imshow(msk)
  # plt.imshow(rawGround)
  plt.show()

  plt.axis('off')
  # plt.imshow(combined)
  # plt.imshow(raw)
  # plt.imshow(msk)
  plt.imshow(rawGround)
  plt.show()

# References


1.   http://deeplearning.net/tutorial/unet.html
2.   https://github.com/ldenoue/keras-unet

